# 交易訊號生成器(美股)第五代(穩定版)：VAR-LSTM預測(強化學習修正) ＋ 獨立新聞分析 + %KD線

## 載入套件

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
import platform
import os
import gc
from datetime import datetime
import subprocess
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from collections import defaultdict
from stockNameList import stock_dict
from tech_index import tech_indi_cal, ma_cal
from senti_pipeline import sentiment_analysis_en, senti_direction_en
from api_encry import decrypt_file

## 設定全域變數及交易股票代號

In [ ]:
aapl = stock_dict['Apple']
msft = stock_dict['微軟']
nvda = stock_dict['輝達']
tsla = stock_dict['Tesla']
goog = stock_dict['Google']
meta = stock_dict['FB']

In [ ]:
signals_dict = defaultdict(list)
change_dict = {} # 收集各股價實際變幅百分比
encrypted_file_path = 'config_enc.env'
key_file_path = 'encryption_key.txt'
api_token = None

## API連線

### 連接

In [ ]:
def api_connect():
    global api_token
    # 解密加密檔案並讀取解密後的內容
    api_token = decrypt_file(encrypted_file_path, key_file_path)

    # 在主程式中使用 api_token 的內容
    if api_token:
        print("已成功解密。")
        print(" ")
        pass
    else:
        print("解密後的內容無效或未找到 API_TOKEN。")

### 中斷

In [ ]:
def api_clear():
    global api_token
    api_token = None
    gc.collect()
    print(f"API狀態：{api_token}")

## 股價資料爬取

In [ ]:
def get_stock_price(df):
    csv_file = f"{df}.csv"
    if not os.path.exists(csv_file):
        # 若檔案不存在，建立空的資料框並儲存為 CSV 檔案
        empty_df = pd.DataFrame(columns=['Date','Open','High','Low','Close','Volume'])
        empty_df.to_csv(csv_file, index=False, encoding='utf-8-sig')
    data = pd.read_csv(csv_file, encoding='utf-8-sig')

    if data.empty:
        # 若資料框為空，下載全部資料
        stock_name = yf.Ticker(df)
        new_data = stock_name.history(period='max', actions=False)
    else:
        # 獲取已有資料的最後日期
        last_date = data["Date"].iloc[-1]

        # 查找有效的交易日日期
        valid_dates = pd.bdate_range(start=last_date, periods=2)[1:]

        # 下載新資料的起始日期為有效的交易日日期
        start_date_str = valid_dates.strftime("%Y-%m-%d")[0]

        # 下載新資料
        stock_name = yf.Ticker(df)
        new_data = stock_name.history(start=start_date_str, end=None, actions=False)

    # 重設行編號
    new_data.reset_index(inplace=True)

    # 合併新資料到已有資料框
    updated_data = pd.concat([data, new_data])

    # 刪除索引欄
    updated_data.to_csv(csv_file, encoding="utf-8-sig", index=False)

    last_updated_date = updated_data["Date"].iloc[-1]
    last_updated_date = pd.to_datetime(last_updated_date).strftime("%Y-%m-%d")
    print(f"{df}股價資料已更新至：", last_updated_date)

In [ ]:
get_stock_price(aapl)
get_stock_price(msft)
get_stock_price(goog)
get_stock_price(nvda)
get_stock_price(tsla)
get_stock_price(meta)

## 更新股價預測模型

In [ ]:
def update_model(df): # 三個模型全部更新最少要6分鐘
    file_prefix = f"{df}_forecast_v"

    # 判定當前作業系統
    system = platform.system()
    if system == 'Windows':
        opsys = 'Windows'
    elif system == 'Darwin':
        opsys = 'Mac OS'
    else:
        opsys = 'Unknown'
    
    print("目前作業系統", opsys)

    # 獲取當前目錄下所有檔案
    files = os.listdir()

    deleted_files = []

    for file in files:
        if file.startswith(file_prefix):
            os.remove(file)
            deleted_files.append(file)

    if deleted_files:
        message = f"已刪除包含'{file_prefix}'的以下模型檔案：\n{deleted_files}"
    else:
        message = f"找不到任何包含'{file_prefix}'的模型檔案"

    print(message)
    # subprocess.run(["python3", "var_lstm_forAutoTrade.py", df], capture_output=True, text=True, check=True)
    if opsys == 'Mac OS': # MAC版
        subprocess.run(["python3", "var_lstm_forAutoTrade_v2.py", df], capture_output=True, text=True, check=True)
    elif opsys == 'Windows': # WIN版
        subprocess.run(["python", "var_lstm_forAutoTrade_v2.py", df], capture_output=True, text=True, check=True)
    else:
        print("錯誤：操作系統不明。")
        raise Exception
    print(f"{df}之預測模型已更新。")

In [ ]:
# 全套6隻更新平均要8-15分鐘
update_model(aapl)
update_model(msft)
update_model(goog)
update_model(nvda)
update_model(tsla)
update_model(meta)

## 交易訊號生成

### 新聞情緒分析

In [ ]:
def senti_pipeline_en(df):
    # 讀取 CSV 檔案
    input_df = pd.read_csv(f"{df}_news.csv", encoding='utf-8-sig')
    # 創建 tokenizer 物件
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

    # 獲取預訓練模型
    model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

    result_df = sentiment_analysis_en(input_df, tokenizer, model)
    result_df.to_csv(f"{df}_news.csv", index=False, encoding='utf-8-sig')

In [ ]:
def signal_news_en(df):
    global signals_dict
    
    # 讀取 CSV 檔案
    df_news = pd.read_csv(f"{df}_news.csv")
    # 呼叫副程式並取得結果
    emotion_result = senti_direction_en(df_news) # 英文模型

    # 列印結果
    print(f"====={df}新聞情緒方向結果=====")
    print("正面數量：", emotion_result[0])
    print("中性數量：", emotion_result[1])
    print("負面數量：", emotion_result[2])
    print("正面比率：", f'{emotion_result[3]:.2f}%')
    print("整體趨勢：", emotion_result[4])
    print("###########################")
    print("")
    
    signals_dict[df].append(emotion_result)

In [ ]:
senti_pipeline_en(aapl)
signal_news_en(aapl)
senti_pipeline_en(msft)
signal_news_en(msft)
senti_pipeline_en(goog)
signal_news_en(goog)
senti_pipeline_en(nvda)
signal_news_en(nvda)
senti_pipeline_en(tsla)
signal_news_en(tsla)
senti_pipeline_en(meta)
signal_news_en(meta)
# print(signals_dict)

### 技術指標

In [ ]:
def signal_tech(df):
    global api_token
    global signals_dict
    global change_dict

    ############################################################
    #######################VAR-LSTM模型預測######################
    ############################################################

    # 載入已訓練好的模型
    model = load_model(f"{df}_forecast_v{datetime.now().date().strftime('%Y_%m_%d')}.h5")

    # 載入測試資料
    test_data = pd.read_csv(f"{df}.csv")
    test_data['Date'] = pd.to_datetime(test_data['Date'])
    test_data.set_index('Date', inplace=True)

    # 將測試資料進行正規化
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(test_data)

    # 將測試資料轉換成模型輸入格式
    X_test = normalized_data.reshape((normalized_data.shape[0], 1, normalized_data.shape[1]))

    # 取得最新的實際股價變化方向
    actual_directions = np.sign(test_data['Close'].diff().iloc[-1])

    # 預測測試資料的股價變化方向
    predicted_directions = model.predict(X_test, verbose=0)

    predicted_close_current = scaler.inverse_transform(predicted_directions)[-1][3]
    predicted_close_prev = scaler.inverse_transform(predicted_directions)[-2][3]

    # 將預測值的最新一天減去前一天的預測值
    # predicted_directions = np.sign(predicted_directions[-1][0] - predicted_directions[-2][0])
    predicted_directions = np.sign(predicted_close_current - predicted_close_prev)

    # 比較預測值變化方向和實際變化方向
    # direction_match = (predicted_directions == actual_directions)   
    actual_directions = 1 if test_data['Close'].diff().iloc[-1] > 0 else (-1 if test_data['Close'].diff().iloc[-1] < 0 else 0)
    predicted_directions = 1 if predicted_directions > 0 else (-1 if predicted_directions < 0 else 0)

    # ############################################################
    # ###########################技術指標##########################
    # ############################################################

    # 獲取最新的收盤價
    current_close = test_data['Close'].iloc[-1]
    prev_close = test_data['Close'].iloc[-2]

    # 預測值變幅
    predicted_change = (((predicted_close_current - predicted_close_prev) / predicted_close_prev) * 100)
    # 實際值變幅
    actual_change = (((current_close - prev_close) / prev_close) * 100)
    # 預測實際變幅差
    if abs(predicted_change) > abs(actual_change):
        change_diff = ((abs(predicted_change) / abs(actual_change)) - 1) * 100
    elif abs(predicted_change) < abs(actual_change):
        change_diff = ((abs(actual_change) / abs(predicted_change)) - 1) * 100
    else:
        change_diff = 0
    
    change_dict[df] = actual_change

    # 呼叫副程式計算指標
    df_tech_indi = tech_indi_cal(test_data)
    df_tech_ma = ma_cal(test_data)
     
    # 取得最新交易日及上個交易日的加權移動平均
    
    sma3_c = df_tech_ma['sma_3'].iloc[-1]
    sma3_p = df_tech_ma['sma_3'].iloc[-2]
    ema5_c = df_tech_ma['ema_5'].iloc[-1]
    ema5_p = df_tech_ma['ema_5'].iloc[-2]
    ema10_c = df_tech_ma['ema_10'].iloc[-1]
    ema10_p = df_tech_ma['ema_10'].iloc[-2]
    ema20_c = df_tech_ma['ema_20'].iloc[-1]
    ema20_p = df_tech_ma['ema_20'].iloc[-2]
    ema30_c = df_tech_ma['ema_30'].iloc[-1]
    ema30_p = df_tech_ma['ema_30'].iloc[-2]
    ema60_c = df_tech_ma['ema_60'].iloc[-1]
    ema60_p = df_tech_ma['ema_60'].iloc[-2]    
    ema90_c = df_tech_ma['ema_90'].iloc[-1]
    ema90_p = df_tech_ma['ema_90'].iloc[-2]    
    
    # 抽調其他指標
    macd_c = df_tech_indi['trend_macd'].iloc[-1]
    macd_p = df_tech_indi['trend_macd'].iloc[-2]
    bb_h_c = df_tech_indi['volatility_bbh'].iloc[-1]
    bb_m_c = df_tech_indi['volatility_bbm'].iloc[-1]
    bb_l_c = df_tech_indi['volatility_bbl'].iloc[-1]
    bb_h_exceed_c = df_tech_indi['volatility_bbhi'].iloc[-1]
    bb_l_exceed_c = df_tech_indi['volatility_bbli'].iloc[-1]
    cmf_c = df_tech_indi['volume_cmf'].iloc[-1]
    
    # RSI指標
    rsi_c = df_tech_indi['momentum_rsi'].iloc[-1]
    # 短RSI指標
    srsi_c = (df_tech_indi['momentum_stoch_rsi'].iloc[-1]) * 100
    
    # %K%D線
    # 交叉條件：K從下穿D=買、K從上穿D＝賣
    # 雙低：KD皆低於20=買、KD皆高於80=賣
    k_c = (df_tech_indi['momentum_stoch_rsi_k'].iloc[-1]) * 100
    k_p = (df_tech_indi['momentum_stoch_rsi_k'].iloc[-2]) * 100
    d_c = (df_tech_indi['momentum_stoch_rsi_d'].iloc[-1]) * 100
    d_p = (df_tech_indi['momentum_stoch_rsi_d'].iloc[-2]) * 100
    
    ############################################################
    ###########################列印資訊##########################
    ############################################################

    print(f"####################################################")
    print(f"###################股票編號：{df} ###################")
    print(f"####################################################")
    print(" ")
    print("====================技術指標數值====================")
    print("===================================================")
    print(" ")
    print("********************實際股價變動********************")
    print("實際最新收盤價：", f"{current_close:.4f}")
    print("實際前日收盤價：", f"{prev_close:.4f}")
    print("實際值變化方向：", actual_directions)
    print("實際股價變化：", f"{test_data['Close'].diff().iloc[-1]:.4f}")
    print("實際變化百分比：", f"{actual_change:.2f}%")
    print("********************預測股價變動********************")
    print("預測最新收盤價：", f"{predicted_close_current:.4f}")
    print("預測前日收盤價：", f"{predicted_close_prev:.4f}")
    print("預測值變化方向：", predicted_directions)
    print("預測股價變化：", f"{predicted_close_current - predicted_close_prev:.4f}")
    print("預測變化百分比：", f"{predicted_change:.2f}%")
    print("********************主要技術指標********************")
    # print("預測實際誤差絕對值：", f"{abs(change_diff):.2f}%")
    print("預測實際誤差絕對值：", f"{(abs(actual_change) - abs(predicted_change)):.2f}%")
    print("加權移動平均線(10日)：", f"{ema10_c:.4f}")
    print("加權移動平均線(30日)：", f"{ema30_c:.4f}")
    print("加權移動平均線(60日)：", f"{ema60_c:.4f}")
    print("加權移動平均線(90日)：", f"{ema90_c:.4f}")
    print("布林上軌：", f"{bb_h_c:.4f}")
    print("布林中線：", f"{bb_m_c:.4f}")
    print("布林下軌：", f"{bb_l_c:.4f}")
    # print("蔡金資金流量指標(-0.25=超賣　+0.25=超買)：", f"{cmf_c:.4f}")
    # print("最新MACD值：", f"{macd_c:.4f}")
    # print("前日MACD值：", f"{macd_p:.4f}")
    print("長RSI值(超賣=30　超買=70)：", f"{rsi_c:.4f}")
    print("短RSI值(超賣=20　超買=80)：", f"{srsi_c:.4f}")
    print("前日%K值(超賣=20　超買=80)：", f"{k_p:.4f}")
    print("前日%D值(超賣=20　超買=80)：", f"{d_p:.4f}")
    print("最新%K值(超賣=20　超買=80)：", f"{k_c:.4f}")
    print("最新%D值(超賣=20　超買=80)：", f"{d_c:.4f}")
    print("最新3日移動平均線：", f"{sma3_c:.4f}")
    print("前日3日移動平均線：", f"{sma3_p:.4f}")
    print(" ")

    print("======================條件檢定======================")
    print("===================================================")
    print(" ")
    print("**********************買入條件**********************")
    print("預測上漲且方向一致：", (predicted_directions == actual_directions) and (predicted_directions == 1))
    print("預測漲幅 > 0.5% - 4%：", predicted_change > 0.5 and predicted_change <= 4)
    # print("預測實際誤差絕對值 < 50%：", (abs(change_diff) < abs(50)))
    print("實際變幅不低於預測變幅：", abs(actual_change) - abs(predicted_change) >= 0)
    print("%K從下穿上%D：", (k_c > d_c) and (k_p < d_p))
    print("%KD皆低於20：", (k_c < 20) and (d_c < 20))
    # print("短RSI < 0.2", srsi_c < 20)
    # print("3日移動平均上升", sma3_c > sma3_p)
    print("**********************賣出條件**********************")
    print("(強條件)方向一致且漲幅皆 > 4%：", (predicted_directions == actual_directions and predicted_directions == 1 and predicted_change > 4 and actual_change > 4))
    print("方向一致且預測跌幅 > 1.5%：", (predicted_directions == actual_directions and predicted_directions == -1 and predicted_change < -1.5))
    # print("預測實際誤差絕對值 < 50%：", (abs(change_diff) < abs(50)))
    print("實際變幅不低於預測變幅：", abs(actual_change) - abs(predicted_change) >= 0)
    print("%K從上穿下%D：", (k_c < d_c) and (k_p > d_p))
    print("%KD皆高於80：", (k_c > 80) and (d_c > 80))
    # print("短RSI > 0.8", srsi_c > 80)
    # print("3日移動平均下降", sma3_c < sma3_p)
    print(" ")

    # print("======================其他資訊======================")
    
    # print("移動平均線(5日)上升：", ema5_c > ema5_p)
    # print("移動平均線(10日)上升：", ema10_c > ema10_p)
    # print("移動平均線(20日)上升：", ema20_c > ema20_p)
    # print("移動平均線(30日)上升：", ema30_c > ema30_c)
    # print("移動平均線(60日)上升：", ema60_c > ema60_p)
    # print("移動平均線(90日)上升：", ema90_c > ema90_c)
    # print(" ")

    ############################################################
    ###########################交易訊號##########################
    ############################################################

    # buy_signal = ((predicted_directions == actual_directions) and (predicted_directions == 1)) \
    #             and (predicted_change > 1) \
    #             and abs(change_diff) < abs(50)
    # sell_signal = ((predicted_directions == actual_directions) and (predicted_directions == -1)) \
    #             and (predicted_change < -1.5) \
    #             and abs(change_diff) < abs(50)

    buy_signal = ((predicted_directions == actual_directions) and (predicted_directions == 1)) \
                and (predicted_change > 0.5 and predicted_change <= 4) \
                and abs(actual_change) - abs(predicted_change) >= 0 \
                and (((k_c > d_c) and (k_p < d_p)) or ((k_c < 20) and (d_c < 20)))
                # and ((srsi_c < 20) and (sma3_c > sma3_p))
    
    sell_signal = (predicted_directions == actual_directions and predicted_directions == 1 and predicted_change > 4 and actual_change > 4) \
                or ((predicted_directions == actual_directions and predicted_directions == -1 and predicted_change < -1.5) \
                and abs(actual_change) - abs(predicted_change) >= 0 \
                and (((k_c < d_c) and (k_p > d_p)) or ((k_c > 80) and (d_c > 80))))
    # sell_signal = ((predicted_directions == actual_directions and predicted_directions == -1 and predicted_change < -1.5) or (predicted_directions == actual_directions and predicted_directions == 1 and predicted_change > 4)) \
    #             and abs(actual_change) - abs(predicted_change) >= 0 \
    #             and (((k_c < d_c) and (k_p > d_p)) or ((k_c > 80) and (d_c > 80)))
                # and ((srsi_c > 80) and (sma3_c < sma3_p))
    if buy_signal:
        signal = "買入"
    elif sell_signal:
        signal = "賣出"
    else:
        signal = "不進行任何動作"

    signals_dict[df].append(signal)

    print("========================決策========================")
    print("====================================================")
    print(" ")
    dec_msg = ""
    if signal == "買入":
        dec_msg += "建議動作：買入"
    elif signal == "賣出":
        dec_msg += "建議動作：賣出"
    else:
        dec_msg += "建議動作：不進行任何動作"
    print(dec_msg)
    print(" ")
    print("####################################################")
    print("####################################################")
    print(" ")
    print(" ")
    print(" ")

    ############################################################
    ###########################推送建議##########################
    ############################################################
    # 備用：預測實際誤差：{abs(change_diff):.2f}%　　\
    # 移動平均(10日)：{ema10_c:.2f}　　　　　\
    # 移動平均(30日)：{ema30_c:.2f}　　　　　\
    # 移動平均(60日)：{ema60_c:.2f}　　　　　\
    # 移動平均(90日)：{ema90_c:.2f}　　　　　\
    # 短RSI值：{srsi_c:.2f}　　\
    # (超賣=20　超買=80)\
    send_msg = f"{datetime.now().strftime('%Y年%m月%d日%H時%M分')}\
            股票編號：{df}　\
            最新收盤價：{current_close:.2f}\
            前日收盤價：{prev_close:.2f}\
            實際方向：{actual_directions}　　　\
            預測方向：{predicted_directions}　　　\
            實際變幅：{actual_change:.2f}%　\
            預測變幅：{predicted_change:.2f}%　\
            絕對誤差：{(abs(actual_change) - abs(predicted_change)):.2f}%　　　\
            最新移動平均(3日)：{sma3_c:.2f}　　　　　\
            前日移動平均(3日)：{sma3_p:.2f}　　　　　\
            布林上軌：{bb_h_c:.2f}　\
            布林中線：{bb_m_c:.2f}　\
            布林下軌：{bb_l_c:.2f}\
            RSI值：{rsi_c:.2f}　　\
            (超賣=30　超買=70)\
            前日%K值：{k_p:.2f}　\
            前日%D值：{d_p:.2f}　\
            最新%K值：{k_c:.2f}　\
            最新%D值：{d_c:.2f}　\
            (超賣=20　超買=80)\
            {dec_msg}"
    try:
        headers = {
            "Authorization": "Bearer " + api_token,
            "Content-Type": "aaplication/x-www-form-urlencoded"
        }
        payload = {
            'message': send_msg
        }
        r = requests.post(
            "https://notify-api.line.me/api/notify", headers=headers, params=payload
        )
        return

    except Exception as e:
        print("推送失敗：API尚未解密。")
        raise e

In [ ]:
api_connect()
signal_tech(aapl)
signal_tech(msft)
signal_tech(goog)
signal_tech(nvda)
signal_tech(tsla)
signal_tech(meta)
api_clear()

In [ ]:
# print(signals_dict)

## 下單執行

In [ ]:
def decision_exe(df):

    ############################################################
    ###########################推送建議##########################
    ############################################################

    global signals_dict
    global change_dict 

    if signals_dict.get(df)[1] == "買入" \
            and signals_dict.get(df)[0][4] == "正面":
        decision = "買入"
    elif signals_dict.get(df)[1] == "賣出" \
            and signals_dict.get(df)[0][4] == "負面":
        decision = "賣出"
    else:
        decision = "不進行任何動作"
    print(f"{df}下單動作建議：{decision}")

    ############################################################
    ###########################寫入紀錄##########################
    ############################################################

    bt_file = f"{df}_s.csv"

    if not os.path.exists(bt_file):
        # 若檔案不存在，建立空的資料框並儲存為 CSV 檔案
        empty_df = pd.DataFrame(columns=['Date','Open','High','Low','Close','Volume'])
        empty_df.to_csv(bt_file, index=False, encoding='utf-8-sig')
    data = pd.read_csv(bt_file, encoding='utf-8-sig')

    if data.empty:
        # 若資料框為空，下載全部資料
        stock_name = yf.Ticker(df)
        new_data = stock_name.history(period='max', actions=False)
    else:
        # 獲取已有資料的最後日期
        last_date = data["Date"].iloc[-1]

        # 查找有效的交易日日期
        valid_dates = pd.bdate_range(start=last_date, periods=2)[1:]

        # 下載新資料的起始日期為有效的交易日日期
        start_date_str = valid_dates.strftime("%Y-%m-%d")[0]

        # 下載新資料
        stock_name = yf.Ticker(df)
        new_data = stock_name.history(start=start_date_str, end=None, actions=False)

    # 重設行編號
    new_data.reset_index(inplace=True)

    # 合併新資料到已有資料框
    updated_data = pd.concat([data, new_data])

    # 刪除索引欄
    updated_data.to_csv(bt_file, encoding="utf-8-sig", index=False)

    print("測試資料已更新至：", datetime.now().date().strftime("%Y-%m-%d"))


    updated_data_2 = pd.read_csv(bt_file)
    # 根據交易訊號執行交易
    if decision == "買入":
        # 執行市價買入訂單
        updated_data_2.loc[updated_data_2.index[-1], 'Decision'] = "買入"
        print("已更新當日交易紀錄為「買入」")
    elif decision == "賣出":
        # 執行市價賣出訂單
        updated_data_2.loc[updated_data_2.index[-1], 'Decision'] = "賣出"
        print("已更新當日交易紀錄為「賣出」")
    else:
        # 不進行任何動作
        updated_data_2.loc[updated_data_2.index[-1], 'Decision'] = "不進行任何動作"
        print("已更新當日交易紀錄為「不進行任何動作」")

    # 獲取上個交易日的索引
    last_index = updated_data_2.index[-2]

    # 比較收盤價的變化方向和決策是否一致，並寫入['Result']欄位   
    if (change_dict.get(df) > 0.5 and change_dict.get(df) <= 4) and updated_data_2.loc[updated_data_2.index[-2], 'Decision'] == "買入":
        updated_data_2.loc[last_index, 'Result'] = "v"
    elif (change_dict.get(df) < -1.5 or change_dict.get(df) > 4) and updated_data_2.loc[updated_data_2.index[-2], 'Decision'] == "賣出":
        updated_data_2.loc[last_index, 'Result'] = "v"
    elif (change_dict.get(df) <= 0.5 and change_dict.get(df) >= -1.5) and updated_data_2.loc[updated_data_2.index[-2], 'Decision'] == "不進行任何動作":
        updated_data_2.loc[last_index, 'Result'] = "v"
    else:
        updated_data_2.loc[last_index, 'Result'] = "x"
    
    updated_data_2.to_csv(bt_file, encoding="utf-8-sig", index=False)

    ############################################################
    ###########################推送建議##########################
    ############################################################

    send_msg = f"{datetime.now().strftime('%Y年%m月%d日%H時%M分')}\
            股票編號：{df}　　\
            新聞情緒：{signals_dict.get(df)[0][4]}　　　　　　\
            股票策略：　　　　　　{signals_dict.get(df)[1]}　　　　　　\
            最終執行動作：　　　　{decision}"
    try:
        headers = {
            "Authorization": "Bearer " + api_token,
            "Content-Type": "application/x-www-form-urlencoded"
        }
        payload = {
            'message': send_msg
        }
        r = requests.post(
            "https://notify-api.line.me/api/notify", headers=headers, params=payload
        )
        return

    except Exception as e:
        print("推送失敗：API尚未解密。")
        raise e


In [ ]:
print(change_dict)

In [ ]:
api_connect()
decision_exe(aapl)
decision_exe(msft)
decision_exe(goog)
decision_exe(nvda)
decision_exe(tsla)
decision_exe(meta)
api_clear()